# Script to merge results tables from both experiments into one

In [1]:
import os
import fnmatch
import pandas as pd
import csv
import numpy as np
from pathlib import Path
from openpyxl import load_workbook
import seaborn as sns
import matplotlib.pyplot as plt
#sns.set_theme(style="whitegrid")

In [2]:
# Definitions

analysis_path = os.path.dirname(os.getcwd())
path = os.path.dirname(analysis_path)

data_path_1 = f'{analysis_path}/table/stats_table.xlsx'
data_path_2 = f'{analysis_path}/table/stats_table_exp_2.xlsx'


In [3]:
ls_group = ['Veh', 'Har', 'DMT', 'Har + DMT']
sample_loc = ['Cerebellum', 'Front Cort']
molecule = ['3-IAA', '3-IAA', 'DMT', 'DMT', 'DMT-N-Oxide', 'DMT-N-Oxide', 'NMT', 'NMT', 'Harmine', 'Harmine', 'Harmol', 'Harmol']

sample_loc_6 = ['Cerebellum', 'Front Cort', 
               'Cerebellum', 'Front Cort', 
               'Cerebellum', 'Front Cort', 
               'Cerebellum', 'Front Cort', 
               'Cerebellum', 'Front Cort',
               'Cerebellum', 'Front Cort']

## Brain concentrations both experiments

In [4]:
df_latex_1 = pd.read_excel(data_path_1, header=0)
df_latex_2 = pd.read_excel(data_path_2, header=0)

df_latex = df_latex_1.copy()
df_latex.insert(loc=6, column='Veh exp2', value= df_latex_2.iloc[:,2])
df_latex.insert(loc=7, column='Har + DMT exp2', value= df_latex_2.iloc[:,3])


In [5]:
order_compound = ['DMT', '3-IAA', 'DMT-N-Oxide', 'NMT', 'Harmine', 'Harmol']
order_region = ['Front Cort', 'Cerebellum']

df_latex = df_latex.astype({'compound':pd.CategoricalDtype(order_compound, ordered=True), 
                            'region':pd.CategoricalDtype(order_region, ordered=True)})
df_latex = df_latex.sort_values(['compound', 'region'])
df_latex_short = df_latex[(df_latex['compound']=='DMT') | (df_latex['compound']=='3-IAA') | (df_latex['compound']=='Harmine')]

In [6]:
df_latex_short

,compound,region,Veh,Har,DMT,Har + DMT,Veh exp2,Har + DMT exp2
3,DMT,Front Cort,10.8 (8.7),4.8 (4.4),14.5 (13.6),149.5 (123.2),0.0 (0.0),1472.7 (369.2)
2,DMT,Cerebellum,5.9 (6.3),4.1 (5.1),8.7 (3.6),105.1 (89.2),0.0 (0.0),2122.7 (434.2)
1,3-IAA,Front Cort,20.4 (9.5),14.1 (1.2),503.9 (137.7),257.5 (103.6),20.9 (2.1),530.7 (129.8)
0,3-IAA,Cerebellum,13.3 (3.7),11.1 (1.9),398.5 (128.3),194.1 (117.2),49.1 (11.4),652.1 (186.5)
9,Harmine,Front Cort,0.8 (0.6),128.4 (65.0),20.1 (46.0),325.6 (345.6),0.0 (0.0),411.6 (137.8)
8,Harmine,Cerebellum,1.7 (3.0),103.9 (64.8),13.6 (31.0),189.0 (134.3),0.0 (0.0),435.9 (107.4)


In [7]:
 ### for latex table   
    
# replace NaNs with empty string    
df_latex_short = df_latex_short.replace(np.nan, '')
df_brain_stats_latex = pd.DataFrame(df_latex_short['compound'])
df_brain_stats_latex.set_axis(['\textbf{compound}'], axis='columns', inplace=True)
df_brain_stats_latex.insert(loc=1, column='\textbf{region}', value=list(map(lambda x: '{' + x +'}', df_latex_short['region'])))

df_brain_stats_latex.insert(loc=2, column='\textbf{Veh}', value=list(map(lambda x: '{' + x +'}', df_latex_short['Veh'])))
df_brain_stats_latex.insert(loc=3, column='\textbf{Har}', value=list(map(lambda x: '{' + x +'}', df_latex_short['Har'])))
df_brain_stats_latex.insert(loc=4, column='\textbf{DMT}', value=list(map(lambda x: '{' + x +'}', df_latex_short['DMT'])))
df_brain_stats_latex.insert(loc=5, column='\textbf{Har + DMT}', value=list(map(lambda x: '{' + x +'}', df_latex_short['Har + DMT'])))
df_brain_stats_latex.insert(loc=6, column='\textbf{Veh.1}', value=list(map(lambda x: '{' + x +'}', df_latex_short['Veh exp2'])))
df_brain_stats_latex.insert(loc=7, column='\textbf{Har + DMT.1}', value=list(map(lambda x: '{' + x +'}', df_latex_short['Har + DMT exp2'])))

df_brain_stats_latex


/var/folders/lb/nxxqtlbj7yd1c5frnp5_vxk40000gn/T/ipykernel_34493/2429323717.py:6: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df_brain_stats_latex.set_axis(['\textbf{compound}'], axis='columns', inplace=True)


,\textbf{compound},\textbf{region},\textbf{Veh},\textbf{Har},\textbf{DMT},\textbf{Har + DMT},\textbf{Veh.1},\textbf{Har + DMT.1}
3,DMT,{Front Cort},{10.8 (8.7)},{4.8 (4.4)},{14.5 (13.6)},{149.5 (123.2)},{0.0 (0.0)},{1472.7 (369.2)}
2,DMT,{Cerebellum},{5.9 (6.3)},{4.1 (5.1)},{8.7 (3.6)},{105.1 (89.2)},{0.0 (0.0)},{2122.7 (434.2)}
1,3-IAA,{Front Cort},{20.4 (9.5)},{14.1 (1.2)},{503.9 (137.7)},{257.5 (103.6)},{20.9 (2.1)},{530.7 (129.8)}
0,3-IAA,{Cerebellum},{13.3 (3.7)},{11.1 (1.9)},{398.5 (128.3)},{194.1 (117.2)},{49.1 (11.4)},{652.1 (186.5)}
9,Harmine,{Front Cort},{0.8 (0.6)},{128.4 (65.0)},{20.1 (46.0)},{325.6 (345.6)},{0.0 (0.0)},{411.6 (137.8)}
8,Harmine,{Cerebellum},{1.7 (3.0)},{103.9 (64.8)},{13.6 (31.0)},{189.0 (134.3)},{0.0 (0.0)},{435.9 (107.4)}


In [8]:
# Write to file
with open(f'{analysis_path}/table/stats_table_both_experiments.tbl', "w") as tbl:

    format = "l" + \
        "@{\hskip 12pt}" +\
        9*"S[table-format = 2.2]"



    tbl.write(df_brain_stats_latex.to_latex(index=False,
                      escape=False,
                      column_format=format)
            )

/var/folders/lb/nxxqtlbj7yd1c5frnp5_vxk40000gn/T/ipykernel_34493/1614378080.py:10: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tbl.write(df_brain_stats_latex.to_latex(index=False,


## Concentrations DMT in % for both experiments

In [9]:
chem_path_1 = f'{analysis_path}/04_chemistry_table_exp1.xlsx'
chem_path_2 = f'{analysis_path}/04_chemistry_table_exp2.xlsx'

In [10]:
chem_table_1 = pd.read_excel(chem_path_1, header =0)
chem_table_1.insert(loc=0, column='Experiment', value='Exp 1')
chem_table_2 = pd.read_excel(chem_path_2, header =0)
chem_table_2.insert(loc=0, column='Experiment', value='Exp 2')


In [11]:
chem_table_2

,Experiment,Treatment_Group,Region,Total DMT,DMT,3-IAA,NMT,DMT-N-Oxide,% DMT,% 3-IAA,% NMT,% DMT-N-Oxide,ng/g DMT,ng/g 3-IAA,ng/g NMT,ng/g DMT-N-Oxide
0,Exp 2,Har + DMT,front cort,15057.86,11274.56,3722.59,11.15,49.57,74.9,24.7,0.1,0.3,2122.7,652.1,1.9,10.1
1,Exp 2,Har + DMT,cerbellum,11003.99,7822.12,3029.31,20.66,131.90,71.1,27.5,0.2,1.2,1472.7,530.7,3.6,26.9
2,Exp 2,Har + DMT,average,13030.92,9548.34,3375.95,15.90,90.73,73.3,25.9,0.1,0.7,1797.7,591.4,2.8,18.5


In [12]:
chem_table = pd.concat([chem_table_1, chem_table_2], ignore_index=True)

order_exp = ['Exp 1', 'Exp 2']
order_group = ['DMT', 'Har + DMT']
order_region = ['front cort', 'cerbellum', 'average']

chem_table = chem_table.astype({'Experiment':pd.CategoricalDtype(order_exp, ordered=True), 
                            'Treatment_Group':pd.CategoricalDtype(order_group, ordered=True),
                            ' Region':pd.CategoricalDtype(order_region, ordered=True),})
chem_table = chem_table.sort_values(['Experiment', 'Treatment_Group', ' Region'], ignore_index=True)
chem_table = chem_table.astype(str)

In [13]:
chem_table_short = pd.DataFrame() #(columns=['Experiment', 'Group', 'Region', 'DMT', '3-IAA', 'NMT', 'DMT-N-oxide', 'Total molar concentration'])

chem_table_short.insert(loc=0, column='Experiment', value=chem_table['Experiment'])
chem_table_short.insert(loc=1, column='Group', value=chem_table['Treatment_Group'])
chem_table_short.insert(loc=2, column='Region', value=chem_table[' Region'])
chem_table_short.insert(loc=3, column='DMT', value=list(map(lambda x,y,z: x + ' (' + y +' — '+ z+'%)', chem_table['ng/g DMT'], chem_table['DMT'], chem_table['% DMT'])))
chem_table_short.insert(loc=4, column='3-IAA', value=list(map(lambda x,y,z: x + ' (' + y +' — '+ z+'%)', chem_table['ng/g 3-IAA'], chem_table['3-IAA'], chem_table['% 3-IAA'])))
chem_table_short.insert(loc=5, column='NMT', value=list(map(lambda x,y,z: x + ' (' + y +' — '+ z+'%)', chem_table['ng/g NMT'], chem_table['NMT'], chem_table['% NMT'])))
chem_table_short.insert(loc=6, column='DMT-N-Oxide', value=list(map(lambda x,y,z: x + ' (' + y +' — '+ z+'%)', chem_table['ng/g DMT-N-Oxide'], chem_table['DMT-N-Oxide'], chem_table['% DMT-N-Oxide'])))
chem_table_short.insert(loc=7, column='Total mol. conc.', value=np.round(chem_table['Total DMT'].astype(float), decimals=0))
chem_table_short['Total mol. conc.']=chem_table_short['Total mol. conc.'].astype(int)

In [14]:
chem_table_short

,Experiment,Group,Region,DMT,3-IAA,NMT,DMT-N-Oxide,Total mol. conc.
0,Exp 1,DMT,front cort,14.5 (77.08 — 2.6%),503.9 (2876.36 — 97.3%),0.0 (0.16 — 0.0%),0.7 (3.43 — 0.1%),2957
1,Exp 1,DMT,cerbellum,8.7 (45.98 — 2.0%),398.5 (2274.61 — 97.9%),0.0 (0.15 — 0.0%),0.6 (3.16 — 0.1%),2324
2,Exp 1,DMT,average,11.6 (61.53 — 2.3%),451.2 (2575.49 — 97.5%),0.0 (0.15 — 0.0%),0.7 (3.29 — 0.1%),2640
3,Exp 1,Har + DMT,front cort,149.5 (793.92 — 34.7%),257.5 (1469.74 — 64.2%),0.2 (0.89 — 0.0%),5.0 (24.54 — 1.1%),2289
4,Exp 1,Har + DMT,cerbellum,105.1 (557.98 — 32.9%),194.1 (1107.88 — 65.3%),0.2 (1.38 — 0.1%),6.2 (30.49 — 1.8%),1698
5,Exp 1,Har + DMT,average,127.3 (675.95 — 33.9%),225.8 (1288.81 — 64.7%),0.2 (1.14 — 0.1%),5.6 (27.52 — 1.4%),1993
6,Exp 2,Har + DMT,front cort,2122.7 (11274.56 — 74.9%),652.1 (3722.59 — 24.7%),1.9 (11.15 — 0.1%),10.1 (49.57 — 0.3%),15058
7,Exp 2,Har + DMT,cerbellum,1472.7 (7822.12 — 71.1%),530.7 (3029.31 — 27.5%),3.6 (20.66 — 0.2%),26.9 (131.9 — 1.2%),11004
8,Exp 2,Har + DMT,average,1797.7 (9548.34 — 73.3%),591.4 (3375.95 — 25.9%),2.8 (15.9 — 0.1%),18.5 (90.73 — 0.7%),13031


In [15]:
chem_table_short.to_excel(f'{analysis_path}/table/chemistry_table_both_clean.xlsx', index=False)

In [20]:
### for latex table     

chem_table_short_latex = pd.DataFrame()
chem_table_short_latex.insert(loc=0, column='\textbf{Experiment}', value=list(map(lambda x: '{' + x +'}', chem_table['Experiment'])))
chem_table_short_latex.insert(loc=1, column='\textbf{Group}', value=list(map(lambda x: '{' + x +'}', chem_table['Treatment_Group'])))
chem_table_short_latex.insert(loc=2, column='\textbf{Region}', value=list(map(lambda x: '{' + x +'}', chem_table[' Region'])))
chem_table_short_latex.insert(loc=3, column='\textbf{DMT}', value=list(map(lambda x,y: '{' + x + ' (' + y +'\%)' +'}', chem_table['DMT'], chem_table['% DMT'])))
chem_table_short_latex.insert(loc=4, column='\textbf{3-IAA}', value=list(map(lambda x,y: '{' + x + ' (' + y +'\%)' +'}', chem_table['3-IAA'], chem_table['% 3-IAA'])))
chem_table_short_latex.insert(loc=5, column='\textbf{NMT}', value=list(map(lambda x,y: '{' + x + ' (' + y +'\%)' +'}', chem_table['NMT'], chem_table['% NMT'])))
chem_table_short_latex.insert(loc=6, column='\textbf{DMT-N-Oxide}', value=list(map(lambda x,y: '{' + x + ' (' + y +'\%)' +'}', chem_table['DMT-N-Oxide'], chem_table['% DMT-N-Oxide'])))
chem_table_short_latex.insert(loc=7, column='\textbf{Total mol. conc.}', value=list(map(lambda x: '{' + x +'}', np.round(chem_table['Total DMT'].astype(float), decimals=0).astype(int).astype(str))))
#chem_table_short_latex['\textbf{Total mol. conc.']=chem_table_short_latex['\textbf{Total mol. conc.}'].astype(int)



In [21]:
chem_table_short_latex

,\textbf{Experiment},\textbf{Group},\textbf{Region},\textbf{DMT},\textbf{3-IAA},\textbf{NMT},\textbf{DMT-N-Oxide},\textbf{Total mol. conc.}
0,{Exp 1},{DMT},{front cort},{77.08 (2.6\%)},{2876.36 (97.3\%)},{0.16 (0.0\%)},{3.43 (0.1\%)},{2957}
1,{Exp 1},{DMT},{cerbellum},{45.98 (2.0\%)},{2274.61 (97.9\%)},{0.15 (0.0\%)},{3.16 (0.1\%)},{2324}
2,{Exp 1},{DMT},{average},{61.53 (2.3\%)},{2575.49 (97.5\%)},{0.15 (0.0\%)},{3.29 (0.1\%)},{2640}
3,{Exp 1},{Har + DMT},{front cort},{793.92 (34.7\%)},{1469.74 (64.2\%)},{0.89 (0.0\%)},{24.54 (1.1\%)},{2289}
4,{Exp 1},{Har + DMT},{cerbellum},{557.98 (32.9\%)},{1107.88 (65.3\%)},{1.38 (0.1\%)},{30.49 (1.8\%)},{1698}
5,{Exp 1},{Har + DMT},{average},{675.95 (33.9\%)},{1288.81 (64.7\%)},{1.14 (0.1\%)},{27.52 (1.4\%)},{1993}
6,{Exp 2},{Har + DMT},{front cort},{11274.56 (74.9\%)},{3722.59 (24.7\%)},{11.15 (0.1\%)},{49.57 (0.3\%)},{15058}
7,{Exp 2},{Har + DMT},{cerbellum},{7822.12 (71.1\%)},{3029.31 (27.5\%)},{20.66 (0.2\%)},{131.9 (1.2\%)},{11004}
8,{Exp 2},{Har + DMT},{average},{9548.34 (73.3\%)},{3375.95 (25.9\%)},{15.9 (0.1\%)},{90.73 (0.7\%)},{13031}


In [22]:
# Write to file
with open(f'{analysis_path}/table/chemistry_table_both_clean.tbl', "w") as tbl:

    format = "l" + \
        "@{\hskip 12pt}" +\
        6*"S[table-format = 2.2]"



    tbl.write(chem_table_short_latex.to_latex(index=False,
                      escape=False,
                      column_format=format)
            )

/var/folders/lb/nxxqtlbj7yd1c5frnp5_vxk40000gn/T/ipykernel_34493/389938714.py:10: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tbl.write(chem_table_short_latex.to_latex(index=False,
